In [1]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

load_dotenv()

dbname=os.getenv("DB_NAME"),

print(dbname)
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    print("Connection established successfully!")
except Exception as e:
    print("Connection failed:", e)


cursor = conn.cursor()

cursor.execute("SELECT * FROM potholes")

# Fetch results (if applicable)
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
cursor.close()
conn.close()

('HotPot',)
Connection established successfully!


In [2]:
#Changing the first_reported_date column to just date
for index, row in df.iterrows():
    df['first_reported_date'] =  pd.to_datetime(df['first_reported_date']).dt.date

In [3]:
#df with just the pothole_id, pothole_size and first_reported_date
df_one = df[['pothole_id', 'pothole_size', 'first_reported_date']]
df_one


,pothole_id,pothole_size,first_reported_date
0,53,Small,2024-07-03
1,54,Small,2024-07-03
2,55,Small,2024-07-03
3,67,Small,2024-07-17
4,73,Djjd,2024-07-18
5,74,Zbbzz,2024-07-18
6,75,Timeeee,2024-07-18
7,43,Large,2024-06-19
8,41,Medium,2024-06-19
9,42,Medium,2024-06-19


In [4]:
#First response from Gemini with order of pothole_id
# json_data = [
#   {
#     "pothole_id": 43
#   },
#   {
#     "pothole_id": 48
#   },
#   {
#     "pothole_id": 41
#   },
#   {
#     "pothole_id": 42
#   },
#   {
#     "pothole_id": 56
#   },
#   {
#     "pothole_id": 40
#   },
#   {
#     "pothole_id": 44
#   },
#   {
#     "pothole_id": 45
#   },
#   {
#     "pothole_id": 53
#   },
#   {
#     "pothole_id": 54
#   },
#   {
#     "pothole_id": 55
#   }
# ]

### For part 2 getting the distances and times from the openRouteService API

##### Format for api is {"coordinates":[[8.681495,49.41461],[8.686507,49.41943],[8.687872,49.420318]]}

In [5]:
import json

def setupCoordinates(json_data):

    data = json.loads(json.dumps(json_data))

    order = []
    for item in data:
        order.append(item['pothole_id'])

    print(order)

    #Sorting the df_twoPrep by the order using the order list as the index
    df_twoPrep = df[['pothole_id', 'coordinates']]
    df_twoPrep = df_twoPrep.set_index('pothole_id').loc[order].reset_index()
    df_twoPrep

    #iterating through the df_twoPrep and create a new list of coordinates, swapping the lat and long
    coordinates = []
    for index, row in df_twoPrep.iterrows():
        coordinates.append(row['coordinates'].split(','))
        coordinates[index] = [float(i) for i in coordinates[index]]
        coordinates[index] = [coordinates[index][1], coordinates[index][0]]
    
    return coordinates, order

#coordinates, order = setupCoordinates(json_data)

In [6]:
ORS_API_KEY = os.getenv("ORS_KEY")

import requests
def getDistanceTime(coordinates):

    #set the headers and the payload
    headers = {
        'Content-Type': 'application/json; charset=utf-8',
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': ORS_API_KEY
    }

    payload = {
        'coordinates': coordinates
    }

    #make the request
    response = requests.post('https://api.openrouteservice.org/v2/directions/driving-car/json', headers=headers, json=payload)
    response.json()

    distance = []
    duration = []

    for i in range(len(response.json()['routes'][0]['segments'])):
        distance.append(response.json()['routes'][0]['segments'][i]['distance'])
        duration.append(response.json()['routes'][0]['segments'][i]['duration'])

    distance
    duration

    duration.append(0)
    distance.append(0)

    return distance, duration

#distance, duration = getDistanceTime(coordinates)


In [7]:
def convertDistanceTime(distance, duration, order, df):
    df_three = df
    df_three = df_three.set_index('pothole_id').loc[order].reset_index()
    df_three['distance (meters)'] = distance
    df_three['duration (seconds)'] = duration
    #df_three

    df_three['distance (KM)'] = df_three['distance (meters)'] / 1000
    df_three['duration (MINS)'] = df_three['duration (seconds)'] / 60
    return df_three

#df_three = convertDistanceTime(distance, duration, order, df_one)

In [8]:
# df_json = df_three.to_json(orient='records')
# df_json

# Gemini Prompt 1

In [9]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""

import os

import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[]
)

def df_to_markdown(df):
    """Converts a pandas DataFrame to a Markdown table string."""
    return df.to_markdown()


prompt = f"""
I have a dataset of potholes with the following columns:

* pothole_id: Unique identifier for the pothole
* pothole_size: Size of the pothole (Small, Medium, Large)
* first_reported_date: Date the pothole was first reported

Please prioritize the potholes for repair based on the following criteria:
* Large potholes should be fixed within 7 days
* Medium potholes should be fixed within 14 days
* Small potholes should be fixed within 30 days

I don't want code, I need you to internally prioritize and output the list of pothole IDs in the following JSON format:

[
  {{"pothole_id": <pothole_id1>}},
  {{"pothole_id": <pothole_id2>}},
  ...
]

Here's the dataset:
{df_to_markdown(df_one)}
"""

response = chat_session.send_message(prompt)

print(response.text)

c:\Users\eazaz\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```json
[
  {"pothole_id": 48},
  {"pothole_id": 56},
  {"pothole_id": 76},
  {"pothole_id": 43},
  {"pothole_id": 41},
  {"pothole_id": 42},
  {"pothole_id": 53},
  {"pothole_id": 54},
  {"pothole_id": 55},
  {"pothole_id": 40},
  {"pothole_id": 44},
  {"pothole_id": 45},
  {"pothole_id": 67}
]
```


In [10]:
response_text = response.text
def textToJson(response_text):
    start_index = response_text.find('json') + len('json\n')
    end_index = response_text.find('```', start_index)
    json_str = response_text[start_index:end_index].strip()
    pothole_data = json.loads(json_str)
    return pothole_data


pothole_data = textToJson(response_text)
pothole_data

[{'pothole_id': 48},
 {'pothole_id': 56},
 {'pothole_id': 76},
 {'pothole_id': 43},
 {'pothole_id': 41},
 {'pothole_id': 42},
 {'pothole_id': 53},
 {'pothole_id': 54},
 {'pothole_id': 55},
 {'pothole_id': 40},
 {'pothole_id': 44},
 {'pothole_id': 45},
 {'pothole_id': 67}]

# Gemini Prompt 2

In [11]:
coordinates, order = setupCoordinates(pothole_data)
distance, duration = getDistanceTime(coordinates)
df_three = convertDistanceTime(distance, duration, order, df_one)

[48, 56, 76, 43, 41, 42, 53, 54, 55, 40, 44, 45, 67]


In [12]:
# from datetime import date, timedelta
# today = date.today()
# tomorrow = today + timedelta(days = 1)

In [13]:
from datetime import date, timedelta

def getNextWorkingDay():
    tomorrow = date.today() + timedelta(days = 1)
    tomorrowDay = tomorrow.weekday() 
    if tomorrowDay == 5:
        tomorrow = tomorrow + timedelta(days = 2)
    elif tomorrowDay == 6:
        tomorrow = tomorrow + timedelta(days = 1)
    return tomorrow

tomorrow = getNextWorkingDay()
print(tomorrow)

2024-07-19


In [14]:
prompt2 = f"""
I have a dataset of potholes with details including size, reported date, distance to the next pothole, and travel time between them. Additionally, I have information about the road crew:

Crew size: 1
Fix times:
    Small pothole: 10 minutes
    Medium pothole: 15 minutes
    Large pothole: 20 minutes
Work hours: 8:00 AM - 4:00 PM (with a 1-hour lunch break from 12:00 PM - 1:00 PM)
Work days: Monday to Friday
Assuming the crew starts work on {tomorrow} at 8:00 AM, I would like you to determine the estimated fix date for each pothole. Please consider travel time between potholes(using duration), work hours, and lunch break when calculating the estimated fix date.

I don't want code, I need you to internally determine the estimated fix date for each pothole and output the list of pothole IDs in the following JSON format:

[
  {{"pothole_id": <pothole_id1>,  "EstimatedFixDate": Date}},
  {{"pothole_id": <pothole_id2>,  "EstimatedFixDate": Date}},
  ...
]

Here's the dataset:
{df_to_markdown(df_three)}
"""

response2 = chat_session.send_message(prompt2)

print(response2.text)

```json
[
  {
    "pothole_id": 48,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 56,
    "EstimatedFixDate": "2024-07-22"
  },
  {
    "pothole_id": 76,
    "EstimatedFixDate": "2024-07-23"
  },
  {
    "pothole_id": 43,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 41,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 42,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 53,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 54,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 55,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 40,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 44,
    "EstimatedFixDate": "2024-07-19"
  },
  {
    "pothole_id": 45,
    "EstimatedFixDate": "2024-07-24"
  },
  {
    "pothole_id": 67,
    "EstimatedFixDate": "2024-07-26"
  }
]
``` 



In [15]:
estimated_fix_dates = textToJson(response2.text)
estimated_fix_dates

[{'pothole_id': 48, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 56, 'EstimatedFixDate': '2024-07-22'},
 {'pothole_id': 76, 'EstimatedFixDate': '2024-07-23'},
 {'pothole_id': 43, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 41, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 42, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 53, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 54, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 55, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 40, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 44, 'EstimatedFixDate': '2024-07-19'},
 {'pothole_id': 45, 'EstimatedFixDate': '2024-07-24'},
 {'pothole_id': 67, 'EstimatedFixDate': '2024-07-26'}]

# Save to DB

In [16]:
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    print("Connection established successfully!")
except Exception as e:
    print("Connection failed:", e)


cursor = conn.cursor()

update_query = "UPDATE potholes SET estimated_fix_date = %s WHERE pothole_id = %s"

#both wokr the same to add date
data = [(item['EstimatedFixDate'], item['pothole_id']) for item in estimated_fix_dates]
#data = [(datetime.strptime(item['EstimatedFixDate'], '%Y-%m-%d').date(), item['pothole_id'],) for item in estimated_fix_dates]

#Set to Null again
#data = [(None, item['pothole_id'],) for item in estimated_fix_dates]

try:
    cursor.executemany(update_query, data)
    conn.commit()  
    print("Records updated successfully!")
except Exception as e:
    print("Failed to update records:", e)
    conn.rollback() 

cursor.close()
conn.close()

Connection established successfully!
Records updated successfully!
